# Temp model comparison

#### **Import python packages**

In [1]:
import pandas as pd
import numpy as np
import cobra
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.offsetbox import (TextArea, DrawingArea, OffsetImage,
                                  AnnotationBbox)
from matplotlib.cbook import get_sample_data
import matplotlib.image as mpimg
import matplotlib.cm as cm
import scipy
import sys
from scipy.stats import linregress
from sklearn.metrics import r2_score

SourceDir = '../src'
sys.path.append(SourceDir)
from utils import *

output_dir = '../plots/'
from plot import *

#### **Function to compare solutions**

In [2]:
def combine_dfs(df_1, df_2, df_1_name, df_2_name):
    combined_df = df_1.copy()
    
    # change the name of flux in old df to 'old_flux'
    combined_df.rename(columns={'flux': df_1_name}, inplace=True)
    
    # add new flux to combined_df with name 'new_flux'
    df_2_fluxes = []
    for rxn_id in combined_df.reaction_id:
        if rxn_id in list(df_2.reaction_id):
            df_2_flux_value = float(df_2[df_2.reaction_id == rxn_id].flux)
            df_2_fluxes.append(df_2_flux_value)
        else:
            df_2_fluxes.append('no rxn') 
    combined_df[df_2_name] = df_2_fluxes
    
    return combined_df

#### **Load Genome Scale Models**

In [3]:
old_model = cobra.io.read_sbml_model("../old_files/Biodesign_grcs/GSMs/Ropacus_annotated_curated.xml")
new_model = cobra.io.read_sbml_model("../models/Ropacus_annotated_curated.xml")

display(old_model)
display(new_model)

Name,ropacus_annotated_curated
Memory address,0x07f4d102f4290
Number of metabolites,1587
Number of reactions,2392
Number of groups,0
Objective expression,1.0*Growth - 1.0*Growth_reverse_699ae
Compartments,"cytosol, periplasm, extracellular space"


Name,ropacus_annotated_curated
Memory address,0x07f4cf03e3450
Number of metabolites,1952
Number of reactions,3021
Number of groups,0
Objective expression,1.0*Growth - 1.0*Growth_reverse_699ae
Compartments,"cytosol, periplasm, extracellular space"


#### **Block meta pathway in old model**

In [4]:
old_model.reactions.get_by_id('CAT23DOX').upper_bound = 0.0
old_model.reactions.get_by_id('CAT23DOX')

Reaction identifier,CAT23DOX
Name,CAT23DOX
Memory address,0x07f4cf539f9d0
Stoichiometry,catechol_c + o2_c --> 2hmcnsad_c Catechol + O2 --> 2-Hydroxymuconate semialdehyde
GPR,WP_005237880_1
Lower bound,0.0
Upper bound,0.0


#### **Run Phenol FBA**

In [5]:
old_phenol_fba_solution =  FBA_pred(old_model, 'phenol') 
new_phenol_fba_solution =  FBA_pred(new_model, 'phenol') 

{'EX_h2o_e': 1000,
 'EX_h_e': 1000,
 'EX_cl_e': 1000,
 'EX_pi_e': 1000,
 'EX_nh4_e': 1000,
 'EX_fe3_e': 1000,
 'EX_k_e': 1000,
 'EX_ca2_e': 1000,
 'EX_mg2_e': 1000,
 'EX_mn2_e': 1000,
 'EX_cobalt2_e': 1000,
 'EX_zn2_e': 1000,
 'EX_cu2_e': 1000,
 'EX_o2_e': 1000,
 'EX_fe2_e': 1000,
 'EX_mobd_e': 1000,
 'EX_so4_e': 1000,
 'EX_phenol_e': 100}

{'EX_ca2_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_fe3_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_mobd_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_zn2_e': 1000.0,
 'EX_phenol_e': 100}

#### **Convert solutions to dataframes**

In [6]:
old_phenol_fba_df = fba_solution_to_df(old_model, old_phenol_fba_solution)
new_phenol_fba_df = fba_solution_to_df(new_model, new_phenol_fba_solution)

#### **Compare exchange reactions**

In [7]:
exchange_rxns = list(new_model.medium.keys())
exchange_rxns.extend(['EX_phenol_e','Growth', 'Growth_Glucose', 'Growth_Phenol'])

old_exchange_df = old_phenol_fba_df[old_phenol_fba_df.reaction_id.isin(exchange_rxns)]
new_exchange_df = new_phenol_fba_df[new_phenol_fba_df.reaction_id.isin(exchange_rxns)]

combined_exchange_df = combine_dfs(old_exchange_df, new_exchange_df, 'old_flux', 'new_flux')

combined_exchange_df

,reaction_id,reaction_name,reaction_reaction,old_flux,new_flux
2152,EX_glc__D_e,R_EX_glc__D_e,glc__D_e <=>,0.000000,0.000000
2153,EX_h2o_e,R_EX_h2o_e,h2o_e <=>,223.507158,279.176508
2155,EX_h_e,R_EX_h_e,h_e <=>,66.819962,18.233076
2165,EX_cl_e,R_EX_cl_e,cl_e <=>,-0.044731,-0.012173
2177,EX_pi_e,R_EX_pi_e,pi_e <=>,-11.054672,-3.008523
2183,EX_nh4_e,R_EX_nh4_e,nh4_e <=>,-77.985794,-21.223794
2201,EX_fe3_e,R_EX_fe3_e,fe3_e <=>,-0.067101,-0.033967
2208,EX_k_e,R_EX_k_e,k_e <=>,-1.677544,-0.456543
2220,EX_ca2_e,R_EX_ca2_e,ca2_e <=>,-0.044731,-0.012173
2223,EX_mg2_e,R_EX_mg2_e,mg2_e <=>,-0.074554,-0.020290


#### **Run Glucose FBA**

In [8]:
old_glucose_fba_solution =  FBA_pred(old_model, 'glucose') 
new_glucose_fba_solution =  FBA_pred(new_model, 'glucose') 

{'EX_glc__D_e': 100,
 'EX_h2o_e': 1000,
 'EX_h_e': 1000,
 'EX_cl_e': 1000,
 'EX_pi_e': 1000,
 'EX_nh4_e': 1000,
 'EX_fe3_e': 1000,
 'EX_k_e': 1000,
 'EX_ca2_e': 1000,
 'EX_mg2_e': 1000,
 'EX_mn2_e': 1000,
 'EX_cobalt2_e': 1000,
 'EX_zn2_e': 1000,
 'EX_cu2_e': 1000,
 'EX_o2_e': 1000,
 'EX_fe2_e': 1000,
 'EX_mobd_e': 1000,
 'EX_so4_e': 1000}

{'EX_ca2_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_fe3_e': 1000.0,
 'EX_glc__D_e': 100,
 'EX_h2o_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_mobd_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_zn2_e': 1000.0}

#### **Convert solutions to dataframes**

In [9]:
old_glucose_fba_df = fba_solution_to_df(old_model, old_glucose_fba_solution)
new_glucose_fba_df = fba_solution_to_df(new_model, new_glucose_fba_solution)

#### **Compare exchange reactions**

In [10]:
exchange_rxns = list(new_model.medium.keys())
exchange_rxns.extend(['EX_phenol_e','Growth', 'Growth_Glucose', 'Growth_Phenol'])

old_exchange_df = old_glucose_fba_df[old_glucose_fba_df.reaction_id.isin(exchange_rxns)]
new_exchange_df = new_glucose_fba_df[new_glucose_fba_df.reaction_id.isin(exchange_rxns)]

combined_exchange_df = combine_dfs(old_exchange_df, new_exchange_df, 'old_flux', 'new_flux')

combined_exchange_df

,reaction_id,reaction_name,reaction_reaction,old_flux,new_flux
2152,EX_glc__D_e,R_EX_glc__D_e,glc__D_e <=>,-100.000000,-100.000000
2153,EX_h2o_e,R_EX_h2o_e,h2o_e <=>,472.528531,692.048739
2155,EX_h_e,R_EX_h_e,h_e <=>,64.239996,286.868251
2165,EX_cl_e,R_EX_cl_e,cl_e <=>,-0.060805,-0.022933
2177,EX_pi_e,R_EX_pi_e,pi_e <=>,-25.892415,-9.765509
2183,EX_nh4_e,R_EX_nh4_e,nh4_e <=>,-89.311942,-33.684635
2201,EX_fe3_e,R_EX_fe3_e,fe3_e <=>,-0.091213,-0.063988
2208,EX_k_e,R_EX_k_e,k_e <=>,-2.280356,-0.860052
2220,EX_ca2_e,R_EX_ca2_e,ca2_e <=>,-0.060805,-0.022933
2223,EX_mg2_e,R_EX_mg2_e,mg2_e <=>,-0.101345,-0.038223


#### **Add Oxidative Phosphorylation Reactions from old model to new model**

In [11]:
rxns_to_add = ['CBFC2pp', 'NDH1_4pp', 'CYO1bpp_syn', 'CYTBDpp']

for rxn_id in rxns_to_add:
    rxn = old_model.reactions.get_by_id(rxn_id)
    
    # do not include genes from the broad genome in the new model
    rxn.gene_reaction_rule = ''
    display(rxn)
    
    new_model.add_reactions([rxn])

Reaction identifier,CBFC2pp
Name,Cytochrome b6/f complex periplasm
Memory address,0x07f4cf53a7950
Stoichiometry,2.0 ficytc6_p + 2.0 h_c + pqh2_p --> 2.0 focytc6_p + 4.0 h_p + pq_p 2.0 Ferricytochrome c6 + 2.0 H+ + Plastoquinol --> 2.0 Ferrocytochrome c6 + 4.0 H+ + Plastoquinone
GPR,
Lower bound,0.0
Upper bound,1000.0


Reaction identifier,NDH1_4pp
Name,Active co2 transporter facilitator (periplasm)
Memory address,0x07f4cf2a04e50
Stoichiometry,co2_p + h2o_c + 3.0 h_c + nadph_c + pq_p --> 3.0 h_p + hco3_c + nadp_c + pqh2_p CO2 + H2O + 3.0 H+ + Nicotinamide adenine dinucleotide phosphate - reduced + Plastoquinone --> 3.0 H+ + Bicarbonate + Nicotinamide adenine dinucleotide phosphate + Plastoquinol
GPR,
Lower bound,0.0
Upper bound,1000.0


Reaction identifier,CYO1bpp_syn
Name,"Cytochrome c oxidase, synechocystis (2 protons)"
Memory address,0x07f4cf52e6d50
Stoichiometry,2.0 focytc6_p + 4.0 h_c + 0.5 o2_p --> 2.0 ficytc6_p + h2o_p + 2.0 h_p 2.0 Ferrocytochrome c6 + 4.0 H+ + 0.5 O2 --> 2.0 Ferricytochrome c6 + H2O + 2.0 H+
GPR,
Lower bound,0.0
Upper bound,1000.0


Reaction identifier,CYTBDpp
Name,Cytochrome oxidase bd (ubiquinol-8: 2 protons) (periplasm)
Memory address,0x07f4cf52fcc10
Stoichiometry,2.0 h_c + 0.5 o2_c + q8h2_c --> h2o_c + 2.0 h_p + q8_c 2.0 H+ + 0.5 O2 + Ubiquinol-8 --> H2O + 2.0 H+ + Ubiquinone-8
GPR,
Lower bound,0.0
Upper bound,1000.0


#### **Run Phenol FBA**

In [12]:
old_phenol_fba_solution =  FBA_pred(old_model, 'phenol') 
new_phenol_fba_solution =  FBA_pred(new_model, 'phenol') 

{'EX_h2o_e': 1000,
 'EX_h_e': 1000,
 'EX_cl_e': 1000,
 'EX_pi_e': 1000,
 'EX_nh4_e': 1000,
 'EX_fe3_e': 1000,
 'EX_k_e': 1000,
 'EX_ca2_e': 1000,
 'EX_mg2_e': 1000,
 'EX_mn2_e': 1000,
 'EX_cobalt2_e': 1000,
 'EX_zn2_e': 1000,
 'EX_cu2_e': 1000,
 'EX_o2_e': 1000,
 'EX_fe2_e': 1000,
 'EX_mobd_e': 1000,
 'EX_so4_e': 1000,
 'EX_phenol_e': 100}

{'EX_ca2_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_fe3_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_mobd_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_zn2_e': 1000.0,
 'EX_phenol_e': 100}

#### **Convert solutions to dataframes**

In [13]:
old_phenol_fba_df = fba_solution_to_df(old_model, old_phenol_fba_solution)
new_phenol_fba_df = fba_solution_to_df(new_model, new_phenol_fba_solution)

#### **Compare exchange reactions**

In [14]:
exchange_rxns = list(new_model.medium.keys())
exchange_rxns.extend(['EX_phenol_e','Growth', 'Growth_Glucose', 'Growth_Phenol'])

old_exchange_df = old_phenol_fba_df[old_phenol_fba_df.reaction_id.isin(exchange_rxns)]
new_exchange_df = new_phenol_fba_df[new_phenol_fba_df.reaction_id.isin(exchange_rxns)]

combined_exchange_df = combine_dfs(old_exchange_df, new_exchange_df, 'old_flux', 'new_flux')

combined_exchange_df

,reaction_id,reaction_name,reaction_reaction,old_flux,new_flux
2152,EX_glc__D_e,R_EX_glc__D_e,glc__D_e <=>,0.000000,0.000000
2153,EX_h2o_e,R_EX_h2o_e,h2o_e <=>,223.507158,271.746031
2155,EX_h_e,R_EX_h_e,h_e <=>,66.819962,-52.312878
2165,EX_cl_e,R_EX_cl_e,cl_e <=>,-0.044731,-0.056541
2177,EX_pi_e,R_EX_pi_e,pi_e <=>,-11.054672,-13.973340
2183,EX_nh4_e,R_EX_nh4_e,nh4_e <=>,-77.985794,-98.575699
2201,EX_fe3_e,R_EX_fe3_e,fe3_e <=>,-0.067101,-0.084817
2208,EX_k_e,R_EX_k_e,k_e <=>,-1.677544,-2.120452
2220,EX_ca2_e,R_EX_ca2_e,ca2_e <=>,-0.044731,-0.056541
2223,EX_mg2_e,R_EX_mg2_e,mg2_e <=>,-0.074554,-0.094238


#### **Run Glucose FBA**

In [15]:
old_glucose_fba_solution =  FBA_pred(old_model, 'glucose') 
new_glucose_fba_solution =  FBA_pred(new_model, 'glucose') 

{'EX_glc__D_e': 100,
 'EX_h2o_e': 1000,
 'EX_h_e': 1000,
 'EX_cl_e': 1000,
 'EX_pi_e': 1000,
 'EX_nh4_e': 1000,
 'EX_fe3_e': 1000,
 'EX_k_e': 1000,
 'EX_ca2_e': 1000,
 'EX_mg2_e': 1000,
 'EX_mn2_e': 1000,
 'EX_cobalt2_e': 1000,
 'EX_zn2_e': 1000,
 'EX_cu2_e': 1000,
 'EX_o2_e': 1000,
 'EX_fe2_e': 1000,
 'EX_mobd_e': 1000,
 'EX_so4_e': 1000}

{'EX_ca2_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_fe3_e': 1000.0,
 'EX_glc__D_e': 100,
 'EX_h2o_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_mobd_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_zn2_e': 1000.0}

#### **Convert solutions to dataframes**

In [16]:
old_glucose_fba_df = fba_solution_to_df(old_model, old_glucose_fba_solution)
new_glucose_fba_df = fba_solution_to_df(new_model, new_glucose_fba_solution)

#### **Compare exchange reactions**

In [17]:
exchange_rxns = list(new_model.medium.keys())
exchange_rxns.extend(['EX_phenol_e','Growth', 'Growth_Glucose', 'Growth_Phenol'])

old_exchange_df = old_glucose_fba_df[old_glucose_fba_df.reaction_id.isin(exchange_rxns)]
new_exchange_df = new_glucose_fba_df[new_glucose_fba_df.reaction_id.isin(exchange_rxns)]

combined_exchange_df = combine_dfs(old_exchange_df, new_exchange_df, 'old_flux', 'new_flux')

combined_exchange_df

,reaction_id,reaction_name,reaction_reaction,old_flux,new_flux
2152,EX_glc__D_e,R_EX_glc__D_e,glc__D_e <=>,-100.000000,-100.000000
2153,EX_h2o_e,R_EX_h2o_e,h2o_e <=>,472.528531,569.199574
2155,EX_h_e,R_EX_h_e,h_e <=>,64.239996,-151.055362
2165,EX_cl_e,R_EX_cl_e,cl_e <=>,-0.060805,-0.067808
2177,EX_pi_e,R_EX_pi_e,pi_e <=>,-25.892415,-28.874663
2183,EX_nh4_e,R_EX_nh4_e,nh4_e <=>,-89.311942,-99.598752
2201,EX_fe3_e,R_EX_fe3_e,fe3_e <=>,-0.091213,-0.101719
2208,EX_k_e,R_EX_k_e,k_e <=>,-2.280356,-2.543003
2220,EX_ca2_e,R_EX_ca2_e,ca2_e <=>,-0.060805,-0.067808
2223,EX_mg2_e,R_EX_mg2_e,mg2_e <=>,-0.101345,-0.113018
